![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fdata-viz-of-the-week&branch=main&subPath=air-quality/air-quality.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Callysto’s Weekly Data Visualization

## Air Quality in Canada

### Recommended Grade levels: 6-12
<br>

### Instructions
#### “Run” the cells to see the graphs
Click “Cell” and select “Run All”.<br> This will import the data and run all the code, so you can see this week's data visualization. Scroll to the top after you’ve run the cells.<br> 

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don’t need to do any coding to view the visualizations**.
The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer? 
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

# Question

Numerous research have suggested possibilities of air pollution exposure leading to detrimental health conditions for humans. [Health Canada](https://www.canada.ca/en/health-canada/services/publications/healthy-living/2021-health-effects-indoor-air-pollution.html) estimates that air pollution contributes to **15,300 deaths per year** in Canada. To fight against air pollution, the [Government of Canada](https://www.canada.ca/en/environment-climate-change/campaigns/canadian-environment-week/clean-air-day/action-air-pollution.html#toc1) launched an ambitious plan, the 2030 Emissions Reduction Plan, which aims to reduce pollutants emissions to net-zero by 2050. In this notebook, we will investigate changes in trends in air pollution by looking at air pollutants emission from 1990 to 2020. Specifically, did the total emission decrease or increase during this period? 

### Goal
Our goal is to investigate the emissions of major air pollutants over the last 30 years. We will use bar graphs to observe changes in total emissions by each year, and use a pie graph to figure out which province in Canada contributes the most to emission of air pollutants. 

In addition to major air pollutants, we also focus on emissions of fine particulate matter (PM2.5) which are particles whose size is less than 2.5 microns (millionth of a meter), or roughly 1/30 the thickness of a human hair. Such fine particles come from human sources such as power plant emissions, car and truck exhaust, cigarettes and tobacco fumes, as well as natural sources such as smoke from wild fires or volcanoes.  These particles can get deep into a person's lungs and may lead to adverse health effects such as premature mortality, increased heart and lung diseases, acute and chronic bronchitis and more. 
   We will analyze the Canadian wildfire data and PM2.5 emissions to see if increased PM2.5 emissions can be attributed to wildfires.

# Gather

### Code:
The code below will import the Python programming libraries we need to gather and organize the data to answer our question.

In [ ]:
## import libraries

%pip install -r requirements.txt
import pyodide_http
pyodide_http.patch_all()

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from functools import partial
import plotly.subplots as sp
import scipy.stats as stats
from tqdm.notebook import tnrange 
from time import sleep
from IPython.display import clear_output

### Data:

In this notebook, we will integrate three different types of datasets: air pollutants emissions, wildfires in Canada, and Canadian population estimates. Original data sources are as follows:
- **Air Pollution Emissions Across Provinces**: [Canada's Air Pollutant Emissions Inventory](https://data.ec.gc.ca/data/substances/monitor/canada-s-air-pollutant-emissions-inventory/APEI_Tables_Canada_Provinces_Territories/?lang)
- **Wildfires in Canada**: [Canadian National Fire Database](https://cwfis.cfs.nrcan.gc.ca/ha/nfdb)
- **Canadian Population Estimates**: [Population Estimates from Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901)<br>
<br>

[Government of Canada](https://www.canada.ca/en/environment-climate-change/services/air-pollution/pollutants/common-contaminants.html) identified sulphur oxides (sulfur oxides), nitrogen oxides, volatile organic compounds, particular matter, carbon monoxide and ammonia, and ground level ozone as the most common air contaminants in Canada. 

[Canada's Air Pollutant Inventory](https://data.ec.gc.ca/data/substances/monitor/canada-s-air-pollutant-emissions-inventory/APEI_Tables_Canada_Provinces_Territories/?lang) has a total of thirteen air pollutants listed. From here, we will focus on sulfur oxides, nitrogen oxides, volatile organic compounds, carbon monoxide, and ammonia; the pollutants that are marked as the most common air contaminants in Canada. Later in this notebook, we will look into particulate matter as well.


### Import the data

Run the following code cells to import air pollutants and population data. For the pollutants data, the data loading progress for each pollutant and each province will be shown by green progress bars. Wait until all the data is loaded and a table including the data is shown.

In [ ]:
# import data :: Air Pollutants 

# create new dataframe
d = {"Province": [],
    "Pollutant": []}
    
provinces = ["AB", "BC", "ON", "MB", "NB", "NS", "YT", "QC", "SK", "NL", "PE"]
common_pollutants = ["NH3","CO","SOX","NOX","VOC"]
provnum = 0

def import_data(province, pollutants):
    df = pd.DataFrame.from_dict(d)
    for i in pollutants:
        p = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/air-quality/data/province-pollutants/" + province + "_" + i + ".csv")
        p_total = p.loc[p["SECTORS"] == "GRAND TOTAL"]
        p_total = p_total.drop(["Unnamed: 1", "Unnamed: 2"], axis=1)
        p_total = p_total.rename(columns={"SECTORS":"Province"})
        p_total.at[1,"Province"] = province
        p_total.insert(1,"Pollutant", i, True)
        df = pd.concat([df, p_total], ignore_index=True)
        
        for i in tnrange(100, desc='Downloading ' + i + ' data for ' + province):
            startRow = 1 + i*100
            sleep(0.01)
    
    global provnum
    if provnum > 11:
        provnum = 0
    else:
        provnum += 1
            
    print("All provincial data for " + province + " has been downloaded." + 
         " Remaining provinces:" + str(len(provinces) - provnum) + "/" + str(len(provinces)) + ".") 
        
    return df

df = pd.concat(map(partial(import_data, pollutants=common_pollutants), provinces),ignore_index=True)
clear_output(wait=True)
df

In [ ]:
# import data :: Population per Capita
per_capita = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/air-quality/data/per-capita.csv")
per_capita

### Comment on the data

[Canada's Air Pollutant Inventory](https://data.ec.gc.ca/data/substances/monitor/canada-s-air-pollutant-emissions-inventory/APEI_Tables_Canada_Provinces_Territories/?lang) datasets include information about air pollutant emissions from all provinces in Canada. Since we need to download multiple Microsoft Excel files to gather all provincial data, we have already completed the **primary data cleaning** here for your convenience. Notice there are 55 rows and 33 columns in the cleaned dataset.

The [Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901) dataset includes information on monthly population estimates from January 1990 to January 2022. 

We will focus our analysis on the data gathered between the years **1990 and 2020 only**. 

_This data is 1794 rows and 14 columns_ 

# Organize

The code below will arrange the data cleanly so that we can analyze it. This is a quality control step for our data which detects anything odd with the data (e.g. structure, missing values), fixes the oddities, and checks if the fixes worked. 

We want to look at provincial emissions of air pollutants *per capita*. To achieve this, first we need to combine the air pollutants and population per capita dataframes that have been already downloaded and imported. To combine the two dataframes, we rename the `GEO` variable in the population per capita dataframe `Province`. 

Once both data are cleaned, we merge the two datasets based on `Year` and `Province`. The resulting dataset will contain provincial air pollutant emissions and population estimates for every year between 1990 and 2020. Then we divide air pollutant emissions `Value` by population `Per Capita` to get emission `By Capita`. 

## Air Pollutants and Population per Capita

In [ ]:
# data cleaning for per captia data
prov = {"Canada": "CAN", "Newfoundland and Labrador": "NL", "Prince Edward Island": "PE",
        "Nova Scotia": "NS", "New Brunswick": "NB", "Quebec": "QC", "Ontario": "ON",
        "Manitoba": "MB", "Alberta": "AB", "Saskatchewan": "SK", "British Columbia": "BC",
        "Yukon": "YK", "Northwest Territories": "NT", "Nunavut": "NU"}

per_capita = per_capita.replace({"GEO": prov})
per_capita["Year"] = pd.to_datetime(per_capita["REF_DATE"]).dt.year
per_capita = per_capita.groupby(["Year", "GEO"], as_index=False)["VALUE"].mean()
per_capita = per_capita.rename(columns={"GEO": "Province", "VALUE": "Per Capita"})
per_capita = per_capita.loc[per_capita["Year"] < 2021]

# melting the original df to convert it from wide to long format
air_pollutant = df.melt(id_vars=["Province", "Pollutant"], var_name="Year", value_name="Value")
# convert year column to type integer type
air_pollutant["Year"] = air_pollutant["Year"].astype(int)
# merge the air pollutant and per capita datasets
pollutant_capita = pd.merge(air_pollutant, per_capita, on=["Year", "Province"])

# divide value by per capita
pollutant_capita["Value"] = pd.to_numeric(pollutant_capita["Value"])
pollutant_capita["By Capita"] = pollutant_capita["Value"].div(pollutant_capita["Per Capita"])
pollutant_capita

In [ ]:
# calculate mean per capita emission by pollutants. 
pollutant_capita_mean = pollutant_capita[["Province", "Pollutant", "By Capita"]].groupby(["Province", "Pollutant"]).mean()
pollutant_capita_mean = pollutant_capita_mean.rename(columns={"By Capita": "Mean"}).reset_index()
pollutant_capita_mean

In [ ]:
# calculate total emission of each pollutants from all provinces.  
pollutant_capita_sum = pollutant_capita[["Province", "Pollutant", "Value"]].groupby(["Province", "Pollutant"]).sum()
pollutant_capita_sum = pollutant_capita_sum.rename(columns={"Value": "Total Emission"}).reset_index()
pollutant_capita_sum

### Comment on the data
Other than looking at air pollution emissions per capita, we also want to look at total amount of air pollution across all provinces. The code above allows us to calculate the cumulative pollutant emissions from 1990 to 2020. 

## Wildfires in Canada

In [ ]:
def organize(df, title):
    df = pd.read_csv(df)
    df = df[df["Jurisdiction"].notna()]
    df = df.melt(id_vars = ["Jurisdiction", "Month"],
                 var_name = "Year",value_name = "Number")
    df = df.fillna(0)
    df = df.groupby(["Jurisdiction"])["Number"].sum()
    df = pd.Series(df).to_frame().reset_index()
    df = df.replace(prov)
    df = df.rename(columns={"Number":str(title),
                                     "Jurisdiction":"Province"})
    return df

firenum = organize("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/air-quality/data/wildfire/total_fire.csv", "Number of Fire")
firearea = organize("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/air-quality/data/wildfire/total_area_burnt.csv", "Area Burned (hectre)")
fire = pd.merge(firenum, firearea, on=["Province"])
fire

### Comment on the data
The [Canadian National Fire Database](https://cwfis.cfs.nrcan.gc.ca/ha/nfdb) has various information on wildfires in Canada. We will look into the total number of wildfires and the total area burned by province. Using this information, we can observe whether `Number of Fire` or `Area Burned (hectre)` correlates better with PM2.5 emissions. 

## Particulate Matter (PM2.5) Emissions

In [ ]:
common_pollutants= ["PM25"]
provnum = 0
pm25 = pd.concat(map(partial(import_data, pollutants=common_pollutants), provinces), ignore_index=True)
pm25 = pm25.melt(id_vars = ["Province", "Pollutant"], var_name = "Year", value_name = "Total Emission")
pm25["Total Emission"] = pd.to_numeric(pm25["Total Emission"])
pm25 = pm25.groupby(["Province"])["Total Emission"].sum()
pm25 = pd.Series(pm25).to_frame().reset_index()
pm25_fire = pd.merge(pm25, fire, on=["Province"])
clear_output(wait=True)
pm25_fire

### Comment on the data
Here we used the same method that we previsouly developed to gather provincial data for air pollutant emissions. To only focus our attention on PM2.5, we trimmed our data to look at total PM2.5 emissions across provinces.

# Explore

First, we will look at a **stacked bar graph** that displays emissions per capita across all provinces. This graph also allows us to observe yearly changes in pollutant emissions. If you want to look at a specific year, drag the button on the slidebar at the bottom of the graph. Additionally, select or unselcect the type of air emission you wish to look at.

In [ ]:
# creating stacked bar graph
stacked_bar = px.bar(pollutant_capita, x="Province", y="By Capita", color="Pollutant", pattern_shape="Pollutant",
                     animation_frame="Year", range_y=[0,1.5])
stacked_bar.update_layout(height=600, width=800, title_text="Provincial Emission per Capita 1990-2020",
                          title_x=0.5)
stacked_bar.update_yaxes(title="Annual Emissions per Capita (tonnes/capita)")
stacked_bar.show()

Second, We will build a **bar graph** to depict cumulative contribution of air pollutants for each province. Click the pollutant drop down menu to select the type of air pollutant you wish to look at. 

In [ ]:
# creating bar graph 
bar_fig = go.Figure()
firstvis = lambda x: True if (x=="CO") else False

for pollutant in pollutant_capita_sum["Pollutant"].unique():
    bar_fig.add_trace(go.Bar(x=pollutant_capita_sum.loc[pollutant_capita_sum["Pollutant"] == str(pollutant)]["Province"],
                             y=pollutant_capita_sum.loc[pollutant_capita_sum["Pollutant"] == str(pollutant)]["Total Emission"],
                             name=str(pollutant),
                             visible=firstvis(pollutant)))

pollutant_buttons = []
num = 0
for p in pollutant_capita_sum["Pollutant"].unique():
    traces = [False] * len(pollutant_capita_sum["Pollutant"].unique())
    if traces[num] == False:
        traces[num] = True
    num += 1 
    pollutant_buttons.append(dict(label=str(p),
                                 method="update",
                                 args=[{"visible":traces}]))

bar_fig.update_layout(showlegend=False,
                      updatemenus=[dict(active=0,
                                        buttons=pollutant_buttons,
                                        direction="down", pad={"r": 10, "t": 10},
                                        showactive=True, x=0.11, xanchor="left", y=1.15, yanchor="top")],
                      legend=dict(yanchor="top", y=1, xanchor="left", x=0.01, bgcolor='rgba(0,0,0,0)'),
                      annotations=[dict(text="Pollutant:",x=0, y=1.09, yref="paper", xref="paper", align="left", showarrow=False)],
                      xaxis_title="Province", yaxis_title="Emission per Capita (tonnes/capita)",
                      title_text="Cumulative Air Pollutants Emission Across Provinces 1990-2020", title_x=0.5, width=800, height=500,
                      hovermode="x unified",barmode="group")

bar_fig.show()

Lastly, we will use a **pie graph** to see provincial contributions of total air pollutant emissions. 

Inquiry questions: 

1. Try to observe if you can see any interesting patterns here. Was there any surprises, explain? 

1. Which province contributed the most with each type of air pollutant?

In [ ]:
# data exploration for creating pie 

pie_fig = go.Figure()

for pollutant in pollutant_capita_mean["Pollutant"].unique():
    pie_fig.add_trace(go.Pie(labels=provinces, values=pollutant_capita_mean.loc[pollutant_capita_mean["Pollutant"] == pollutant]["Mean"],
                             name=pollutant, insidetextorientation='radial', sort=False,
                             visible=firstvis(pollutant)))

pie_fig.update_traces(textposition='inside', textinfo='percent+label', 
                      marker=dict(line=dict(color='#FFFFFF', width=1.5)))

pie_fig.update_layout(showlegend=True,
                      updatemenus=[dict(active=0,
                                        buttons=pollutant_buttons,
                                        direction="down", pad={"r": 10, "t": 10},
                                        showactive=True, x=0.11, xanchor="left", y=1.15, yanchor="top")],
                      legend=dict(yanchor="top", y=1, xanchor="left", x=0.01, bgcolor='rgba(0,0,0,0)'),
                      annotations=[dict(text="Pollutant:",x=0, y=1.09, yref="paper", xref="paper", align="left", showarrow=False)],
                      xaxis_title="Province", yaxis_title="Emission per Capita (tonnes/capita)",
                      title_text="Provincial Mean for Air Pollutant Emission per Capita 1990-2020", title_x=0.5, width=800, height=500)


pie_fig.show()

Now we take a look at the wildfire and PM2.5 data combined. Here, we created subplots of scatterplots to depict any relationships present among the two datasets. We see similar trends in the wildfire data and PM2.5 emission data, as expected. But what does it really mean?

In [ ]:
fire_fig = sp.make_subplots(
    rows=2, cols=1,
    specs=[[{"secondary_y": True}],
          [{"secondary_y": True}]],
    subplot_titles=("Total PM2.5 Emission vs. Total Number of Fire","Total PM2.5 Emission vs. Total Area Burned"),
    vertical_spacing = 0.1)

fire_fig.add_trace(go.Scatter(x=pm25_fire["Province"], y=pm25_fire["Total Emission"], name="Total PM2.5 Emission(tonnes)", showlegend=True, legendgroup="group1"),
                   row=1, col=1, secondary_y=False)
fire_fig.add_trace(go.Scatter(x=pm25_fire["Province"], y=pm25_fire["Number of Fire"], name="Total number of wildfire", showlegend=True, legendgroup="group1"),
                   row=1, col=1, secondary_y=True)
fire_fig.add_trace(go.Scatter(x=pm25_fire["Province"], y=pm25_fire["Total Emission"], name="Total PM2.5 Emission(tonnes)", showlegend=True, legendgroup="group2"),
                   row=2, col=1, secondary_y=False)
fire_fig.add_trace(go.Scatter(x=pm25_fire["Province"], y=pm25_fire["Area Burned (hectre)"], name="Total area burned (hectres)", showlegend=True, legendgroup="group2"),
                   row=2, col=1, secondary_y=True)

fire_fig.update_yaxes(title_text="Total PM2.5 Emission (tonnes)", row=1, col=1, secondary_y=False)
fire_fig.update_yaxes(title_text="Total number of wildfire", row=1, col=1, secondary_y=True)
fire_fig.update_yaxes(title_text="Total PM2.5 Emission (tonnes)", row=2, col=1, secondary_y=False)
fire_fig.update_yaxes(title_text="Total area burned (hectres)", row=2, col=1, secondary_y=True)

fire_fig.update_annotations(font_size=15)

fire_fig.update_layout(hovermode="x unified", font=dict(size=10),title_x = 0.5)

fire_fig.update_layout(
    height=800, 
    width=800, 
    title_text="PM2.5 Emissons Across Provinces", 
    xaxis2_title = "Province",
    legend_tracegroupgap = 305,
    legend=dict(yanchor="top", y=1, xanchor="left", x=0.01, bgcolor='rgba(0,0,0,0)')
)

fire_fig.show()

# Interpret
Here are some of the interesting trends we notice from the above visualizations:
1. Air pollutant emissions have decreased significantly over the last 30 years from all provinces. Saskatchewan emissions are the highest in the country after 30, why do you think that is? 
1. Alberta has been a huge contributor for ammonia emissions, nitrogen oxides, sulphur oxides and volatile organic compounds emissions historically. 
1. The unusually high PM2.5 emissions from Saskatchewan seem to be heavily related to the *total area burnt* by wildfire, whereas high PM2.5 emissions from Alberta seem to be more related to the *total number* of wildfires. So, which one of the two variables are more strongly correlated with PM2.5 emissions?

In [ ]:
# calculating correlational coefficient for the occurance of wildfire and PM2.5 emission.
corr, pval=stats.pearsonr(pm25_fire["Total Emission"],pm25_fire["Area Burned (hectre)"])
print("Correlation coefficient for total area burnt and total PM2.5 emission: " + str(corr))
corr, pval=stats.pearsonr(pm25_fire["Total Emission"],pm25_fire["Number of Fire"])
print("Correlation coefficient for number of fire and total PM2.5 emission: " + str(corr))

The code above confirmed to us that **the total number of wildfires and PM2.5 emissions** have a correlation value of $0.795$, whereas the **the total area burnt by wildfire and PM2.5 emissions** have a correlation value of $0.387$. Therefore, we can conclude that the total area burnt by wildfire is a better predictor for PM2.5 emissions.

# Communicate
Below are some writing prompts to help you reflect on the new information that is presented from the data. When we look at the evidence, think about what you perceive about the information. Is this perception based on what the evidence shows? If others were to view it, what perceptions might they have?

- I used to think ____________________ but now I think ____________________. 
- I wish I knew more about ____________________. 
- This visualization reminds me of ____________________. 
- I really like ____________________.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)